# Os dados

Para coletar os dados de outros países, foram utilizadas as APIs dos sites:

- [About Corona](https://about-corona.net/dashboard/)
- [Europe RestFul](https://restcountries.eu/)

In [1]:

import requests
import pandas as pd

covid_api = 'https://corona-api.com/countries/'
rest_countries = 'https://restcountries.eu/rest/v2/alpha/'
country = 'DE' # Alpha-2 ISO3166

data_json =  requests.get(covid_api + country).json()
country = requests.get(covid_api + country).json()

N = country['data']['population']

print(country['data']['name'])


Germany


### Organizando os dados

In [2]:

from datetime import datetime

df = pd.DataFrame(data_json['data']['timeline'])
df = df.sort_values('date').reset_index()

from datetime import datetime, timedelta
df['date'] = [datetime.fromisoformat(f) for f in df['date']]
df = df.drop_duplicates(subset='date', keep = 'last')

# Criando o vetor de tempo
first_date = df['date'].iloc[0]
size_days = (df['date'].iloc[-1] - df['date'].iloc[0]).days
date_vec = [first_date + timedelta(days=k) for k in range(size_days)]

new_df = pd.DataFrame(date_vec, columns=['date'])
new_df = pd.merge(new_df, df, how='left', on= 'date')
new_df = new_df.drop(columns= ['index',  'updated_at', 'is_in_progress'])

for col in new_df.columns[1:]:
    new_df[col] = new_df[col].interpolate(method='polynomial', order=1)
df = new_df.dropna()

df.head()

,date,deaths,confirmed,active,recovered,new_confirmed,new_recovered,new_deaths
0,2020-01-27,0.0,4.0,4.0,0.0,4.0,0.0,0.0
1,2020-01-28,0.0,4.0,4.0,0.0,0.0,0.0,0.0
2,2020-01-29,0.0,4.0,4.0,0.0,0.0,0.0,0.0
3,2020-01-30,0.0,5.0,5.0,0.0,1.0,0.0,0.0
4,2020-01-31,0.0,8.0,8.0,0.0,3.0,0.0,0.0


### Visualizando os dados

In [3]:

from bokeh.models   import Legend, ColumnDataSource, RangeTool, LinearAxis, Range1d, HoverTool
from bokeh.palettes import brewer, Inferno256
from bokeh.plotting import figure, show
from bokeh.layouts  import column
from bokeh.io       import output_notebook

output_notebook()

import numpy as np

# Criando os valores para legenda no plot
year =  [str(int(d.year)) for d in df['date'] ]
month = [("0"+str(int(d.month)))[-2:] for d in df['date'] ]
day =   [("0"+str(int(d.day)))[-2:] for d in df['date'] ]

# Criando a fonte de dados
source = ColumnDataSource(data={
    'Data'       : df['date'].values,
    'd': day, 'm': month, 'y': year,
    'Infectados Acc' : df['confirmed'].values,
    'Mortes'     : df['deaths'].values,
    'Ativo'     : df['active'].values,
    'Recuperados': df['recovered'].values
})


# Criando a figura
p = figure(plot_height=500,
           plot_width=600,
           x_axis_type="datetime", 
           tools="", 
           #y_axis_type="log",
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Incluindo as curvas
i_p = p.line(x='Data', y='Ativo',
             legend_label="Infectados", 
             line_cap="round", line_width=5, color="#c62828", source=source)
m_p = p.line(x='Data', y='Mortes',
             legend_label="Mortes",
             line_cap="round", line_width=5, color="#512da8", source=source)
c_p = p.line(x='Data', y='Infectados Acc',
             legend_label="Infectados Acc",
             line_cap="round", line_width=5, color="#0288d1", source=source)
r_p = p.line(x='Data', y='Recuperados',
             legend_label="Recuperados",
             line_cap="round", line_width=5, color="#388e3c", source=source)

# Colocando as legendas
p.legend.click_policy="hide"
# p.legend.location = "top_left"
p.legend.location = "top_left"

# Incluindo a ferramenta de hover
p.add_tools(HoverTool(
    tooltips=[
        ( 'Indivíduos', '$y{i}'),
        ( 'Data',       '@d/@m/@y' ),
    ],
    renderers=[
        m_p, i_p, c_p, r_p
    ]
))

show(p)

Loading BokehJS ...

## Verificando os dados

In [4]:

dif_I = np.diff(df['active'])
cum = []
cum.append(dif_I[0])

for k, i in enumerate(dif_I):
    cum.append(cum[-1] + i)
    
cum = np.array(cum)
cum += df['deaths'].to_numpy() + df['recovered'].to_numpy()

print("Erro entre casos acumulados e valores de confirmados: {}".format(
        round(sum((cum - df['confirmed'].values)**2 / len(cum)),2) ) ) 


Erro entre casos acumulados e valores de confirmados: 16.0


## Criando os dados SIR

In [5]:

I = df['active'].to_numpy()
R = df['recovered'].to_numpy()
M = df['deaths'].to_numpy()
S = N - R - I

# Creating the time vector
t = np.linspace(0, len(I), len(I))


Sd, Id, Md, Rd, td = S, I, M, R, t

In [6]:

p = figure(plot_height=500,
           plot_width=600,
           x_axis_type="datetime", 
           tools="", 
           y_axis_type="log",
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Incluindo as curvas
p.line(t, I,
       legend_label="Infectados", 
       line_cap="round", line_width=5, color="#c62828")
p.line(t, M,
       legend_label="Mortes",
       line_cap="round", line_width=5, color="#512da8")
p.line(t, R,
       legend_label="Recuperados",
       line_cap="round", line_width=5, color="#388e3c")

# Colocando as legendas
p.legend.click_policy="hide"
# p.legend.location = "top_left"
p.legend.location = "bottom_right"

p.scatter(td, Id,
          legend_label="Infectados - Reamostrados", 
          fill_color="#e91e63", fill_alpha=0.2, line_width=0, size=4)
p.scatter(td, Rd,
          legend_label="Removidos - Reamostrados", 
          fill_color="#1a237e", fill_alpha=0.2, line_width=0, size=4)

show(p)

In [7]:

ponder = 1
R_neg, I_neg = Rd < 0, Id < 0
M_neg, S_neg = M < 0, Sd < 0

print("R negatives: ", sum(R_neg), " first at ", 
      np.argmax(R_neg)/ponder, " last at ", (len(R_neg) - np.argmax(R_neg[::-1]))/ponder)
print("I negatives: ", sum(I_neg), " first at ", 
      np.argmax(I_neg)/ponder, " last at ", (len(I_neg) - np.argmax(I_neg[::-1]))/ponder)
print("M negatives: ", sum(M_neg), " first at ", 
      np.argmax(M_neg)/ponder, " last at ", (len(M_neg) - np.argmax(M_neg[::-1]))/ponder)
print("S negatives: ", sum(S_neg), " first at ", 
      np.argmax(S_neg)/ponder, " last at ", (len(S_neg) - np.argmax(S_neg[::-1]))/ponder)


R negatives:  0  first at  0.0  last at  125.0
I negatives:  0  first at  0.0  last at  125.0
M negatives:  0  first at  0.0  last at  125.0
S negatives:  0  first at  0.0  last at  125.0


# Estimando utilizando todos os dados

In [14]:

from models import *

dataset = dict(S=Sd, I=Id, R=Rd)

# Create the model
sir_model = ss.SIR(pop=N, focus=["S", "I", "R"])

# Adjust the parameters
sir_model.fit(dataset, td,
              search_pop=True,
              pop_sens=[0.001,0.01],
              beta_sens=[100000,1000], 
              r_sens=[100000,100])


# Predict the model
sim_res = sir_model.predict((Sd[0], Id[0], Rd[0]), td)

	 ├─ S(0) ─ I(0) ─ R(0) ─  [81802253.0, 4.0, 0.0]
	 ├─ beta ─   1   r ─   0.14285714285714285
	 ├─ beta bound ─   1e-05  ─  1000
	 ├─ r bound ─   1.4285714285714284e-06  ─  14.285714285714285
	 ├─ equation weights ─   [0.000890733412929348, 1, 0.44184636282048173]
	 ├─ Running on ─  differential_evolution SciPy Search Algorithm
	 └─ Defined at:  105.332785677135  ─  0.05745242500587899 



In [15]:

print("Parâmetros estimados: ", sir_model.parameters)
print("Suposto Ro: ", sir_model.parameters[0] * sir_model.parameters[-1] / sir_model.parameters[1])
print("Dias contaminados: ", 1 / sir_model.parameters[1])


Parâmetros estimados:  [1.05332786e+02 5.74524250e-02 2.07933761e-03]
Suposto Ro:  3.812239830838294
Dias contaminados:  17.405705675568473


In [16]:

p = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

p.line(t, I,  
       legend_label="Infectados", color="#c2185b", line_width=4)
p.line(t, R,  
       legend_label="Removidos", color="#0288d1", line_width=4)

# Show the results
p.line(td, sim_res[1],  
       legend_label="Infectados - Modelo", line_dash="dashed", color="#5e35b1", line_width=3)
p.line(td, sim_res[2],  
     legend_label="Removidos - Modelo", line_dash="dashed", color="#f57f17", line_width=3)
p.line(td, sim_res[0],  
      legend_label="Susceptible - Modelo", line_dash="dashed", color="#43a047", line_width=3)

show(p)

In [17]:
p = figure(plot_height=500,
           plot_width=600,
           tools="", 
           #y_axis_type="log",
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

p.line(td, sir_model.parameters[-1]*N - Rd - Id ,  
       legend_label="Sucetiveis", color="#c2185b", line_width=4)

# Show the results
p.line(td, sim_res[0],  
      legend_label="Suscetiveis - Modelo", line_dash="dashed", color="#5e35b1", line_width=3)

show(p)

In [18]:

p = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Soma das estimativas - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Show the results
p.line(td, sir_model.parameters[-1]*N*np.ones((len(td),)),  
     legend_label="N - pop total", color="#f57f17", line_width=3)
p.line(td, sim_res[0] + sim_res[1] + sim_res[2],  
       legend_label="Estimados", line_dash="dashed", color="#5e35b1", line_width=3)


show(p)


In [19]:

days = len(t) + 150
t_test = np.linspace(0, days, days)

# Predict the model
sim_res = sir_model.predict([S[0], I[0], R[0]], t_test)



p = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

p.line(t, I,  
       legend_label="Infectados", color="#c2185b", line_width=4)
p.line(t, R,  
       legend_label="Removidos", color="#0288d1", line_width=4)

# Show the results
p.line(t_test, sim_res[0],  
     legend_label="Suscetiveis - Modelo", line_dash="dashed", color="#1565c0", line_width=3)
p.line(t_test, sim_res[1],  
       legend_label="Infectados - Modelo", line_dash="dashed", color="#5e35b1", line_width=3)
p.line(t_test, sim_res[2],  
     legend_label="Removidos - Modelo", line_dash="dashed", color="#f57f17", line_width=3)

show(p)

